# Preprocessing des données

corpus de test : 

In [94]:
import pandas as pd
corpus_sephora3k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/corpus_sephora3k.csv",sep=";", error_bad_lines=False)
corpus_sephora3k.head()
text = corpus_sephora3k[corpus_sephora3k['language'] == 'en']['text'].tolist()

C:\Users\a.tekiouk\AppData\Local\Temp\ipykernel_62504\735203815.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  corpus_sephora3k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/corpus_sephora3k.csv",sep=";", error_bad_lines=False)


#### Identification des doublons
Afin d'éviter d'analyser le texte de post identiques, nous allons ici supprimer les doublons présent dans notre corpus de texte.

Pour ce faire nous allon utiliser la librairie textdistance (`pip install text-distance`), et nottament la distance normalisée `textdistance.hamming.normalized_distance` qui nous permettra de définir un seuil entre 0 et 1.

In [117]:
import textdistance
import numpy as np

In [3]:
hamming = textdistance.hamming.normalized_distance #initialisiation de hamming avec la distance normalisée.

On doit enlever les hashtags temporairement afin de vérifier s'il existe des doublons, car en effet des hashtag différent sur deux posts qui sont en fait identiques peuvent perturber le tri.

Fonction permettant de créer une copy de notre corpus de texte, sans les hashtags : 

In [116]:
import re

def clean_hashtag(t): #--text
    txt = text.copy()
    for i in range(len(txt)):
        txt[i] = re.sub("#[A-Za-z0-9_]+","", txt[i]) #On supprime tout les types de #

In [115]:
hamming(' a a a Je joue foot','Je joue foot a a a')
#Compare mot à mot la chaine de caractère ?
#hamming(text[7],'The tea is HOT! Did you know that these 4 big beauty companies own the vast majority of beauty sales? Were you shocked to see that some of your favorite brands are owned by the same companies?')

0.8947368421052632

Une fois les hashtags retiré, il nous faut calculer la distance entre tout les éléments du corpus. La fonction suivante va calculer la distance entre les élements de la liste (corpus de texte) et ensuite supprimer les élements qui sont présent plusieurs fois :

In [69]:
# /!\ Renvoie une matrice de distance, pas opti
#------------------------------------------------------------------------------------------
def dist_mat(t): #-- text
    dist_matrix = np.zeros((len(t),len(t)),dtype=np.float)
    for i in range(len(t)):
        for j in range(len(t)):
            dist_matrix[i,j] = hamming(t[i],t[j])
    return dist_matrix
#------------------------------------------------------------------------------------------

def del_double(t,s): # --text --seuil (s dans [0;1], deux textes identiques ont une distance de 0)
    i = 0
    r = len(t)
    while(i<r):
        r = len(t)
        j=i+1
        while(j<r):
            if(hamming(t[i],t[j]) <= s ): # Si la distance entre les deux élemens de la liste inf à seuil
                del t[j] #delete
                r = len(t) #on actualise la taille de la listes
            else:
                j+=1
        i+=1
    return t

# Rajouter le texte d'origine(avec les #) et supprimez dans les deux listes, pas opti


Suppression des posts identifiés comme doublons :

In [109]:
del_double(txt,0.4)

1984


In [68]:
print(len(txt))

1735


#### Ratio mot/hashtag
Calcul du ratio mot/hastag dans un post

In [30]:
import spacy 
from spacy.lang.en import English

(OPTIONEL) Ici nous ajoutons le token 'hashtag' dans la pipe du modèle spacy.

In [65]:

#  /!\ Ne marche pas
#--------------------------------------------------------------------------------------------
from spacy.vocab import Vocab
from spacy.language import Language

@Language.component("hashtag")
def hashtag_pipe(doc):
    merged_hashtag = False
    while True:
        for token_index,token in enumerate(doc):
            if token.text == '#':
                if token.head is not None:
                    start_index = token.idx
                    end_index = start_index + len(token.head.text) + 1
                    print(start_index, end_index)
                    with doc.retokenize() as retokenizer:
                        if retokenizer.merge(doc[start_index:end_index]) is not None:
                            merged_hashtag = True
                            break
        if not merged_hashtag:
            break
        merged_hashtag = False
    return doc

nlp.add_pipe("hashtag")

#-----------------------------------------------------------------------------------------------------------

<function __main__.hashtag_pipe(doc)>

Afin de calculer le ration mo/hashtags, nous allons claculer le nombre de mot, cependant en utilisant la pipe spacy classique, un '#' est un token en lui-même. Nous devons donc calculer le nom de de '#' et le soustraire au nombre total de mot, car en effet tout les mot précédé d'un '#' seront considéré comme de "vrai' mots.

In [86]:
from spacy.lang.en import English


def del_junk_post(t,s): # --text --seuil (s dans [0;1], le seuil 1 permet de supprimer un post composé à 100% de hashtag)
    nlp = English()
    i = 0
    r = len(t)
    while(i<r):
        r = len(t)
        print(i)
        nb_hash = 0
        nb_word = 0
        doc = nlp(t[i]) # On tokenize l'element de notre liste
        for token in doc: # Pour chaque token de la liste
            if(token.text == '#'):
                nb_hash+=1
            else:
                nb_word+=1
        print('nb hash : \f', nb_hash)
        print('nb word : \f', nb_word)
        if(nb_word-nb_hash==0): # On vérifie d'abord que le nombre de mot n'est pas nul, auquel cas nous supprimmons le post
            del t[i]
            r = len(t)
        elif(nb_hash/(nb_word-nb_hash) > s): # On calcul le ratio, si inférieur au seuil => on delete
            del t[i]
            r = len(t)
        i+=1
    return t
    


In [93]:
tst = ['a #salut #bonjour #hello', 'Bonjour à tous','je je je je #salut #bonjour #hello','Salut']
del_junk_post(tst,0.2)

Yes
0
1


['Bonjour à tous', 'Salut']

In [78]:
print(tst)

['Bjr  #dsojkncdsjncdsj #dsojkncdsjncdsj #dsojkncdsjncdsj #dsojkncdsjncdsj #dsojkncdsjncdsj #dsojkncdsjncdsj #dsojkncdsjncdsj #dsojkncdsj0ncdsj #dsojkncdsjncdsj #dsojkncdsjncdsj', 'Bjr']


Suppression des post ayant un ratio mot/hashtag trop faible :

#### Fréquence d'apparition dans la langue anglaise
Fréquence d'apparition de chaque mot du post dans la langue anglaise

check les seuils , décomposer les mot dans les hashtag, position du hashtag dans le post, dictionnaire frequence langue anglaise